# AC-Solver Iterative Refinement — Colab Runner

**Runtime recommendation: High-RAM CPU**  
The search (V-guided greedy, beam) is CPU-only. GPU only helps training, which takes ~10 min per iteration — not the bottleneck.

**Persistence:** Repo lives on Google Drive. If Colab disconnects, re-run cells 1–3 (fast), then Cell 6 to resume.

## Cell 1 — Mount Google Drive

In [26]:
from google.colab import drive
drive.mount('/content/drive')
print('Drive mounted.')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive mounted.


## Cell 2 — Clone or Update Repo on Drive

**First time only:** enter your GitHub Personal Access Token when prompted.  
Get one at: GitHub → Settings → Developer Settings → Personal access tokens → Fine-grained → repo read access.  
After first clone, this cell just does `git pull`.

In [27]:
import os
import subprocess

REPO_URL = 'https://github.com/Avi161/AC-Solver-Caltech.git'
DRIVE_DIR = '/content/drive/MyDrive/AC-Solver-Caltech'
BRANCH = 'feat/test-new-exp-claude'

if not os.path.exists(DRIVE_DIR):
    # First time: clone with token
    from getpass import getpass
    token = getpass('GitHub Personal Access Token: ')
    auth_url = REPO_URL.replace('https://', f'https://{token}@')
    subprocess.run(['git', 'clone', auth_url, DRIVE_DIR], check=True)
    subprocess.run(['git', 'checkout', BRANCH], cwd=DRIVE_DIR, check=True)
    print('Repo cloned to Drive.')
else:
    # Already cloned: checkout branch first, then pull
    subprocess.run(['git', 'checkout', BRANCH], cwd=DRIVE_DIR, check=True)
    result = subprocess.run(['git', 'pull'], cwd=DRIVE_DIR, capture_output=True, text=True)
    print(result.stdout or 'Already up to date.')

os.chdir(DRIVE_DIR)
print(f'Working directory: {os.getcwd()}')

Updating bdfc942..5b989fb
Fast-forward
 experiments/colab_iterative_refinement.ipynb | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)

Working directory: /content/drive/MyDrive/AC-Solver-Caltech


## Cell 3 — Install Dependencies

In [28]:
import subprocess
result = subprocess.run(
    ['pip', 'install', '-q', '-r', 'requirements.txt'],
    cwd=DRIVE_DIR, capture_output=True, text=True
)
print(result.stdout[-2000:] if result.stdout else 'Done.')
if result.returncode != 0:
    print('ERRORS:', result.stderr[-1000:])

Done.


## Cell 4 — Verify Setup

In [29]:
import os, json

checks = {
    'Original model checkpoint':  'value_search/checkpoints/best_mlp.pt',
    'Feature stats':              'value_search/checkpoints/feature_stats.json',
    'Greedy solved presentations': 'ac_solver/search/miller_schupp/data/greedy_solved_presentations.txt',
    'Greedy search paths':         'ac_solver/search/miller_schupp/data/greedy_search_paths.txt',
    'Config':                      'experiments/config.yaml',
}

all_ok = True
for label, path in checks.items():
    full = os.path.join(DRIVE_DIR, path)
    exists = os.path.exists(full)
    status = '✓' if exists else '✗  MISSING'
    print(f'  {status}  {label}')
    if not exists:
        all_ok = False

# Check refinement state (if resuming)
state_file = os.path.join(DRIVE_DIR, 'experiments/refinement/refinement_state.json')
if os.path.exists(state_file):
    with open(state_file) as f:
        state = json.load(f)
    print(f'\n  Saved state found: iteration={state["iteration"]}, '
          f'total_solved_history={state["total_solved_per_iteration"]}')
else:
    print('\n  No saved state — will start fresh.')

if all_ok:
    print('\nAll checks passed. Ready to run.')
else:
    print('\nFix missing files before running.')

  ✓  Original model checkpoint
  ✓  Feature stats
  ✓  Greedy solved presentations
  ✓  Greedy search paths
  ✓  Config

  Saved state found: iteration=0, total_solved_history=[533, 533]

All checks passed. Ready to run.


## Cell 5 — Configuration
Edit these before running. Then run Cell 6 (fresh start) or Cell 7 (resume).

In [ ]:
# ── Tune these ────────────────────────────────────────────────────────────────
MAX_ITERATIONS  = 5       # How many search→train cycles
MAX_PATH_LENGTH = 300     # Reject training paths longer than this
ENABLE_MCTS     = False   # MCTS is slow; keep False unless you have many hours
MAX_NODES       = 100_000 # 100K nodes: ~25s per presentation, ~5hrs per iteration
# ──────────────────────────────────────────────────────────────────────────────

import sys
sys.path.insert(0, DRIVE_DIR)

print('Config set:')
print(f'  max_iterations  = {MAX_ITERATIONS}')
print(f'  max_path_length = {MAX_PATH_LENGTH}')
print(f'  enable_mcts     = {ENABLE_MCTS}')
print(f'  max_nodes       = {MAX_NODES:,}')


# ── Progress streaming helper (used by Cells 6 and 7) ─────────────────────────
def run_with_progress(cmd, cwd):
    """Run cmd as subprocess, printing headers normally and progress on one line."""
    import subprocess, sys, os

    HEADER_PATTERNS = [
        '===', 'ITERATION', '--- Step', 'complete:', 'REFINEMENT COMPLETE',
        'ERROR', 'Traceback', 'File "', 'Error:', 'Running:', 'Resuming:',
        'Total presentations', 'Greedy baseline', 'Resumed from',
        'No previous state', 'Loading greedy', 'Converged', 'Interrupted',
        'State saved',
    ]

    env = os.environ.copy()
    env['PYTHONUNBUFFERED'] = '1'  # force line-by-line flushing in subprocess

    proc = subprocess.Popen(
        cmd, cwd=cwd,
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1, env=env,
    )

    last_was_progress = False
    for raw in proc.stdout:
        line = raw.rstrip()
        if not line:
            continue
        is_header = any(p in line for p in HEADER_PATTERNS)
        if is_header:
            if last_was_progress:
                sys.stdout.write('\n')
            print(line)
            last_was_progress = False
        else:
            sys.stdout.write(f'\r  {line:<90}')
            sys.stdout.flush()
            last_was_progress = True

    if last_was_progress:
        sys.stdout.write('\n')
    proc.wait()
    return proc.returncode

## Cell 6 — Clean Stale State (run before Fresh Start)
Run this to wipe any previous refinement state. **Skip if resuming a valid run.**

In [31]:
cmd = [
    sys.executable,
    os.path.join(DRIVE_DIR, 'experiments/iterative_refinement.py'),
    '--max-iterations', str(MAX_ITERATIONS),
    '--max-path-length', str(MAX_PATH_LENGTH),
]
if ENABLE_MCTS:
    cmd.append('--enable-mcts')
if MAX_NODES is not None:
    cmd.extend(['--max-nodes', str(MAX_NODES)])

print('Running:', ' '.join(cmd))
rc = run_with_progress(cmd, cwd=DRIVE_DIR)
print('Exit code:', rc)

Running: /usr/bin/python3 /content/drive/MyDrive/AC-Solver-Caltech/experiments/iterative_refinement.py --max-iterations 5 --max-path-length 800
    AC-Solver Iterative Refinement Pipeline                                                 
    State file:     /content/drive/MyDrive/AC-Solver-Caltech/experiments/refinement/refinement_state.json
  Total presentations: 1190
  Loading greedy-solved paths as seed...
  Greedy baseline: 533 solved
  ITERATION 0
    Model: /content/drive/MyDrive/AC-Solver-Caltech/value_search/checkpoints/best_mlp.pt (original, trained on greedy paths)
  --- Step 1: Search ---
    Searching 657/1190 unsolved presentations                                               
  Running: /usr/bin/python3 /content/drive/MyDrive/AC-Solver-Caltech/experiments/run_experiments.py --config /content/drive/MyDrive/AC-Solver-Caltech/experiments/refinement/iter_0/iter_config.yaml --indices /content/drive/MyDrive/AC-Solver-Caltech/experiments/refinement/iter_0/unsolved_indices.txt
  

KeyboardInterrupt: 

In [ ]:
import os, shutil

refinement_dir = os.path.join(DRIVE_DIR, 'experiments/refinement')
state_file     = os.path.join(refinement_dir, 'refinement_state.json')
paths_file     = os.path.join(refinement_dir, 'all_solved_paths.json')

removed = []
for f in [state_file, paths_file]:
    if os.path.exists(f):
        os.remove(f)
        removed.append(os.path.basename(f))

# Remove stale iter_* subdirs
if os.path.isdir(refinement_dir):
    for name in os.listdir(refinement_dir):
        if name.startswith('iter_') or name.startswith('checkpoints_iter_') or name.startswith('training_data'):
            p = os.path.join(refinement_dir, name)
            shutil.rmtree(p) if os.path.isdir(p) else os.remove(p)
            removed.append(name)

if removed:
    print('Removed:', ', '.join(removed))
else:
    print('Nothing to clean — already fresh.')

In [32]:
import subprocess
result = subprocess.run(['ps', 'aux'], capture_output=True, text=True)
for line in result.stdout.split('\n'):
    if 'python' in line:
        print(line)


root          78  0.3  0.0      0     0 ?        Z    02:37   0:15 [python3] <defunct>
root          79  0.0  0.1  99520 75728 ?        S    02:37   0:01 python3 /usr/local/bin/colab-fileshim.py
root         100  0.1  0.2 655512 151968 ?       Sl   02:37   0:05 /usr/bin/python3 /usr/local/bin/jupyter-server --debug --transport="ipc" --ip=172.28.0.12 --ServerApp.token= --port=9000 --FileContentsManager.root_dir=/ --FileContentsManager.allow_hidden=True --ServerApp.log_format="|%(levelname)s|%(message)s" --ServerApp.iopub_data_rate_limit=1e10 --MappingKernelManager.root_dir=/content
root        4871  0.0  0.0   4364  3128 ?        S    02:56   0:00 bash -c tail -n +0 -F "/root/.config/Google/DriveFS/Logs/drive_fs.txt" | python3 /opt/google/drive/drive-filter.py > "/root/.config/Google/DriveFS/Logs/timeouts.txt" 
root        4873  0.0  0.0  18844 11008 ?        S    02:56   0:00 python3 /opt/google/drive/drive-filter.py
root        7141  0.0  0.2 996176 119800 ?       Ssl  03:05   0:02 /u

## Cell 7 — Resume After Disconnect
After Colab disconnects: re-run Cells 1–3 (takes ~1 min), configure Cell 5, then run this cell.  
It picks up from the last completed iteration automatically.

In [ ]:
cmd = [
    sys.executable,
    os.path.join(DRIVE_DIR, 'experiments/iterative_refinement.py'),
    '--resume',
    '--max-iterations', str(MAX_ITERATIONS),
    '--max-path-length', str(MAX_PATH_LENGTH),
]
if ENABLE_MCTS:
    cmd.append('--enable-mcts')
if MAX_NODES is not None:
    cmd.extend(['--max-nodes', str(MAX_NODES)])

print('Resuming:', ' '.join(cmd))
rc = run_with_progress(cmd, cwd=DRIVE_DIR)
print('Exit code:', rc)

## Cell 8 — Check Progress Anytime

In [ ]:
import os, json

state_file = os.path.join(DRIVE_DIR, 'experiments/refinement/refinement_state.json')
if not os.path.exists(state_file):
    print('No state file yet — run Cell 6 first.')
else:
    with open(state_file) as f:
        state = json.load(f)

    paths_file = os.path.join(DRIVE_DIR, 'experiments/refinement/all_solved_paths.json')
    total_solved = 0
    if os.path.exists(paths_file):
        with open(paths_file) as f:
            total_solved = len(json.load(f))

    print(f'Current iteration:  {state["iteration"]}')
    print(f'Total solved:       {total_solved}/1190')
    print(f'Solved per iter:    {state["solved_per_iteration"]}')
    print(f'Total per iter:     {state["total_solved_per_iteration"]}')

    refinement_dir = os.path.join(DRIVE_DIR, 'experiments/refinement')
    print(f'\nFiles in refinement dir:')
    for f in sorted(os.listdir(refinement_dir)):
        fpath = os.path.join(refinement_dir, f)
        size_mb = os.path.getsize(fpath) / 1e6 if os.path.isfile(fpath) else 0
        tag = f'({size_mb:.1f} MB)' if os.path.isfile(fpath) else '(dir)'
        print(f'  {f}  {tag}')